# Slip Convergence Investigation

On the Setup of a planar angular interface we investigate singularities near the contact-line.  
In dependance of the (slip) boundary conditions there should (or not) occur a singularity at the contact line.  
The reasoning is, that if there is a singularity present, we will not observe optimal convergence.  
However, if the introduction of slip suffices to regularize this singularity, optimal convergence should be recovered.  

The interface is initialized and fixed int time, such that it exhibits a small deflection from the equilibrium contact-angle.  
We then proceed by introducing slip on the fluid-solid interfaces aswell as the fluid-fluid interface.  
Furthermore, the influence of evaporation is then examined. THis serves as an example, but the results are not meaningful

In [ ]:
#r "../../src/L4-application/BoSSSpad/bin/Release/net6.0/BoSSSpad.dll"
//#r "../../src/L4-application/BoSSSpad/bin/Debug/net6.0/BoSSSpad.dll"
using System;
using System.Data;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSFE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [ ]:
BoSSSshell.WorkflowMgm.Init("SlipConvergence");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();

In [ ]:
DataTable data = new DataTable("ConvergenceData");
data.Clear();
data.Columns.Add("Name", typeof(string));
data.Columns.Add("Controls", typeof(List<XNSFE_Control>));
data.Columns.Add("Timesteps", typeof(List<ITimestepInfo>));
data.Columns.Add("SessionID", typeof(List<Guid>));

data.Columns.Add("P-Convergence", typeof(Dictionary<int, double>));
data.Columns.Add("P-PlotData", typeof(Plot2Ddata));

data.Columns.Add("U-Convergence", typeof(Dictionary<int, double>));
data.Columns.Add("U-PlotData", typeof(Plot2Ddata));

data.Columns.Add("V-Convergence", typeof(Dictionary<int, double>));
data.Columns.Add("V-PlotData", typeof(Plot2Ddata));

data.Columns.Add("T-Convergence", typeof(Dictionary<int, double>));
data.Columns.Add("T-PlotData", typeof(Plot2Ddata));

In [ ]:
for(byte fs_slip = 0; fs_slip <= 2; fs_slip++){
    for(byte ff_slip = 0; ff_slip <= 2; ff_slip++){
        foreach(bool evap in new [] {false, true}){
            DataRow row = data.NewRow();
            List<XNSFE_Control> subcontrols = new List<XNSFE_Control>();
            string name = $"WedgeConvergence";
            switch(fs_slip){
                case 0: default: name = name + "_NoSlip"; break;
                case 1: name = name + "_NavierSlip"; break;
                case 2: name = name + "_FreeSlip"; break;
            }
            switch(ff_slip){
                case 0: default: name = name + "_NoSlipInterface"; break;
                case 1: name = name + "_NavierSlipInterface"; break;
                case 2: name = name + "_FreeSlipInterface"; break;
            }
            switch(evap){
                case false: default: name = name + "_Material"; break;
                case true: name = name + "_Evaporation"; break;
            }
            foreach(int res in new [] {1, 2, 3, 4, 5, 6}){
                var C = BoSSS.Application.XNSFE_Solver.HardcodedControl.WedgeConvergence(2, res, fs_slip, ff_slip, evap);
                C.SessionName = name + $"_H{res}";                
                C.ProjectName = "SlipConvergence";
                C.savetodb = true;
                C.SetDatabase(BoSSSshell.WorkflowMgm.DefaultDatabase);
                subcontrols.Add(C);
            }

            row["Name"] = name;
            row["Controls"] = subcontrols;
            data.Rows.Add(row);
        }
    }
}

In [ ]:
data.Rows.Count().Display();

In [ ]:
BoSSSshell.WorkflowMgm.Sessions

Run the simulations

In [ ]:
foreach(DataRow row in data.Rows){
    foreach(XNSFE_Control C in (List<XNSFE_Control>)row["Controls"]){
        try{
            C.FailOnSolverFail = false;
            if(BoSSSshell.WorkflowMgm.Sessions.Where(s => s.Name == C.SessionName).Count() == 0){
                C.Run();
            } else{
                $"{C.SessionName} already computed!".Display();
            }
        } catch (Exception e){
            $"Caught exception for {C.SessionName}".Display();
            e.Display();
        } finally {
            $"Finished {C.SessionName}".Display();
        }
    }
}

Obtain convergence orders

In [ ]:
foreach(DataRow row in data.Rows){
    List<ITimestepInfo> ts = new List<ITimestepInfo>();
    List<Guid> ids = new List<Guid>();

    foreach(XNSFE_Control C in (List<XNSFE_Control>)row["Controls"]){
        var s = BoSSSshell.WorkflowMgm.Sessions.Where(s => s.Name == C.SessionName).First();
        ts.Add(s.Timesteps.Last());
        ids.Add(s.ID);
    }

    row["Timesteps"] = ts;
    row["SessionID"] = ids;

    row["P-PlotData"] = ts.ToEstimatedGridConvergenceData("Pressure", normType: NormType.L2_embedded);
    row["P-Convergence"] = ((Plot2Ddata)row["P-PlotData"]).Regression().ToDictionary(kvp => Convert.ToInt32(kvp.Key), kvp => kvp.Value);

    row["U-PlotData"] = ts.ToEstimatedGridConvergenceData("VelocityX", normType: NormType.L2_embedded);
    row["U-Convergence"] = ((Plot2Ddata)row["U-PlotData"]).Regression().ToDictionary(kvp => Convert.ToInt32(kvp.Key), kvp => kvp.Value);

    row["V-PlotData"] = ts.ToEstimatedGridConvergenceData("VelocityY", normType: NormType.L2_embedded);
    row["V-Convergence"] = ((Plot2Ddata)row["V-PlotData"]).Regression().ToDictionary(kvp => Convert.ToInt32(kvp.Key), kvp => kvp.Value);

    row["T-PlotData"] = ts.ToEstimatedGridConvergenceData("Temperature", normType: NormType.L2_embedded);
    row["T-Convergence"] = ((Plot2Ddata)row["T-PlotData"]).Regression().ToDictionary(kvp => Convert.ToInt32(kvp.Key), kvp => kvp.Value);
}

In [ ]:
String.Format("{6,1} | {0,60} | {1,7} | {2,13} | {3,13} | {4,13} | {5,13}", "Trialname", "Degree", "P-Convergence", "U-Convergence", "V-Convergence", "T-Convergence", "#").Display();
int n = 0;
foreach(DataRow row in data.Rows){
    for(int i = 0; i < ((Dictionary<int, double>)row["U-Convergence"]).Count(); i++){
        string p = String.Format("{0}|{1}|{2}|{3}", ((Dictionary<int, double>)row["P-Convergence"]).Keys.ElementAt(i), ((Dictionary<int, double>)row["U-Convergence"]).Keys.ElementAt(i), ((Dictionary<int, double>)row["V-Convergence"]).Keys.ElementAt(i), ((Dictionary<int, double>)row["T-Convergence"]).Keys.ElementAt(i));
        String.Format("{6,1} | {0,60} | {1,7} | {2,13:0.0#} | {3,13:0.0#} | {4,13:0.0#} | {5,13:0.0#}", row["Name"], p, ((Dictionary<int, double>)row["P-Convergence"]).Values.ElementAt(i), ((Dictionary<int, double>)row["U-Convergence"]).Values.ElementAt(i), ((Dictionary<int, double>)row["V-Convergence"]).Values.ElementAt(i), ((Dictionary<int, double>)row["T-Convergence"]).Values.ElementAt(i), n).Display();       
    }
    n++;
}

In [ ]:
((Plot2Ddata)data.Rows[0]["T-PlotData"]).PlotNow()